In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import torch
from transformers import AutoModel #, AutoModelForMaskedLM

# import pickle
from tqdm import tqdm

import os
import csv
import warnings
warnings.filterwarnings('ignore')


%run hyena_utility.py

%run preprocess_utility.py

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

### Load Human Chrom Sequences from .fa File

In [ ]:
fasta_file = "../genome.hg38rg.fa"
chrom_sequences = read_fasta(fasta_file)

In [ ]:
def Subsequence2Embedding(subsequence):
    tok_seq = tokenizer(subsequence)
    tok_seq = tok_seq["input_ids"]  

    # place on device, convert to tensor
    tok_seq = torch.LongTensor(tok_seq).unsqueeze(0)  # unsqueeze for batch dim
    tok_seq = tok_seq.to(device)

    with torch.inference_mode():
        embeddings = model(tok_seq)

    mean_embeddings = embeddings.mean(dim=1) # Mean across the sequence length dimension
    mean_embeddings = mean_embeddings.squeeze(0)  # This will change the shape to [256]
    
    return mean_embeddings

In [ ]:
def append_rows_to_csv(csv_Filename, rows):
    with open(csv_Filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        for row in rows:
            writer.writerow(row)

### Main Process

In [ ]:
pretrained_model_name = 'hyenadna-medium-160k-seqlen'
model, tokenizer, max_length =  get_model_tokenizer_maxlen(pretrained_model_name)
model.to(device)
model.eval()

In [ ]:
def get_subsequence(chrom_name, start_pos, length):
    
    if chrom_name in chrom_sequences:
        sequence = chrom_sequences[chrom_name]
        subsequence = sequence[start_pos:start_pos + length]
        return subsequence
    else:
        raise ValueError(f"Chromosome '{chrom_name}' not found in the FASTA file.")

In [ ]:
%%time

comp = {'A':1, 'C':2, 'G':3, 'T':4}

max_length = 128

csv_Filename = './homo_sapiens_hyena_embedding.csv'
if os.path.exists(csv_Filename):
    os.remove(csv_Filename)

datafile_path = '../../datasets/task03-homo-sapiens/Homo_sapiens.GRCh38.109.txt.gz'  
df = preprocess_home_sapiens_datafile(datafile_path)


df.loc[df['SIZE'] > max_length, 'END'] = df['START'] + max_length
df.loc[df['SIZE'] > max_length, 'SIZE'] = max_length
df.drop(columns=['END','TYPE','CLUSTER'], inplace=True)
# df


rows=[]
for index, row in df.iterrows():      
    chrom=row['CHROM']
    rowid=row['ROWID']
    pos_start=row['START']

    if pos_start<=1:
        pos_start=1
    y=row['y']
    length = row['SIZE'] # max_length
    
    subsequence = get_subsequence(chrom, pos_start, length)
    if 'N' in subsequence:
        print("The character 'N' is present in the string.")
        
    embedding = Subsequence2Embedding(subsequence)
    # print (embedding)

    # feature=np.array(embedding_df.iloc[64])
    rows.append(np.append(embedding.cpu().numpy(),  [rowid, y]))  #chrom, length,  


    if index > 0 and (index % 2000) == 0:
        append_rows_to_csv(csv_Filename, rows)
        rows=[]
        print (f"index = {index} completed")
        
append_rows_to_csv(csv_Filename, rows)

print(f"Create File: "+csv_Filename)

### Load CSV File

In [ ]:
import pandas as pd

def load_embedding_file(csv_filename):

    df=pd.read_csv(csv_filename)
    
    column_names = [f'{i}' for i in range(1, df.shape[1]-1)]
    column_names.extend(['ROWID', 'y'])
    
    df.columns = column_names
    return df
df = load_embedding_file('./homo_sapiens_hyena_embedding.csv')
df